# Manipulating Data in DataFrames HW


#### Let's get started applying what we learned in the lecure!

I've provided several questions below to help test and expand you knowledge from the code along lecture. So let's see what you've got!

First create your spark instance as we need to do at the start of every project.

In [2]:

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
# May take awhile locally
spark = SparkSession.builder.appName("Manipulate").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

You are working with 1 core(s)


## Read in our Republican vs. Democrats Tweet DataFrame

Attached to the lecture

In [3]:
tweets = spark.read.csv('Datasets/Rep_vs_Dem_tweets.csv',inferSchema=True,header=True)

## About this dataframe

Extracted tweets from all of the representatives (latest 200 as of May 17th 2018)

**Source:** https://www.kaggle.com/kapastor/democratvsrepublicantweets#ExtractedTweets.csv

Use either .show() or .toPandas() check out the first view rows of the dataframe to get an idea of what we are working with.

In [4]:
tweets.limit(5).toPandas()

,Party,Handle,Tweet
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P..."
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Al...
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...
3,"Congress has allocated about $18…""",None,None
4,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...


**Prevent Truncation of view**

If the view you produced above truncated some of the longer tweets, see if you can prevent that so you can read the whole tweet.

In [5]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
tweets.limit(5).toPandas()

,Party,Handle,Tweet
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L"
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages.
3,"Congress has allocated about $18…""",None,None
4,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto . Thanks for taking the time to meet with @LatinoLeader ED Marucci Guzman. #NALCABPolicy2018.…


**Print Schema**

First, check the schema to make sure the datatypes are accurate. 

In [6]:
tweets.printSchema()

root
 |-- Party: string (nullable = true)
 |-- Handle: string (nullable = true)
 |-- Tweet: string (nullable = true)



In [7]:
from pyspark.sql.functions import *
import pyspark.sql.functions as F

## 1. Can you identify any tweet that mentions the handle @LatinoLeader using regexp_extract?

It doesn't matter how you identify the row, any identifier will do. You can test your script on row 5 from this dataset. That row contains @LatinoLeader. 

In [12]:
from pyspark.sql.functions import regexp_replace, regexp_extract
new= tweets.withColumn('extracted',regexp_extract('Tweet','(.)(@LatinoLeader)(.)', 2))
new.limit(5).toPandas()

,Party,Handle,Tweet,extracted
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L",
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…,
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages.,
3,"Congress has allocated about $18…""",None,None,None
4,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto . Thanks for taking the time to meet with @LatinoLeader ED Marucci Guzman. #NALCABPolicy2018.…,@LatinoLeader


## 2. Replace any value other than 'Democrate' or 'Republican' with 'Other' in the Party column.

We can see from the output below, that there are several other values other than 'Democrate' or 'Republican' in the Part column. We are assuming that this is dirty data that needs to be cleaned up.

In [26]:
from pyspark.sql.functions import when
tweets.select("Party",(when(tweets.Party == 'Democrat' , 'Democrate' )\
                       .when(tweets.Party == 'Republican','Republican')\
                              .otherwise('Other'))\
  .alias("Party_updated")).show(5,False)

+----------------------------------+-------------+
|Party                             |Party_updated|
+----------------------------------+-------------+
|Democrat                          |Democrate    |
|Democrat                          |Democrate    |
|Democrat                          |Democrate    |
|Congress has allocated about $18…"|Other        |
|Democrat                          |Democrate    |
+----------------------------------+-------------+
only showing top 5 rows



## 3. Delete all embedded links (ie. "https:....)

For example see the first row in the tweets dataframe. 

*Note: this may require an google search :)*

## 4. Remove any leading or trailing white space in the tweet column

In [28]:
df = tweets.withColumn('Tweet',trim(tweets.Tweet))
df.limit(4).toPandas()

,Party,Handle,Tweet
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L"
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages.
3,"Congress has allocated about $18…""",None,None


## 5. Rename the 'Party' column to 'Dem_Rep'

No real reason here :) just wanted you to get practice doing this. 

In [29]:
renamed = tweets.withColumnRenamed('Party','Dem_Rep')
renamed.limit(4).toPandas()

,Dem_Rep,Handle,Tweet
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L"
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages.
3,"Congress has allocated about $18…""",None,None


## 6. Concatenate the Party and Handle columns

Silly yes... but good practice.

pyspark.sql.functions.concat_ws(sep, *cols)[source] <br>
Concatenates multiple input string columns together into a single string column, using the given separator.

In [44]:
tweets.select(concat_ws(' ', tweets.Party,tweets.Tweet).alias('concat')).limit(4).toPandas()

,concat
0,"Democrat Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L"
1,Democrat RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…
2,Democrat RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages.
3,"Congress has allocated about $18…"""


## Challenge Question

Let's image that we want to analyze the hashtags that are used in these tweets. Can you extract all the hashtags you see?

In [40]:
new= tweets.withColumn('extracted',regexp_extract('Tweet','#(\w+)',1))
new.limit(5).toPandas()


,Party,Handle,Tweet,extracted
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L",SaveTheInternet
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…,
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages.,
3,"Congress has allocated about $18…""",None,None,None
4,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto . Thanks for taking the time to meet with @LatinoLeader ED Marucci Guzman. #NALCABPolicy2018.…,NALCABPolicy2018


# Let's create our own dataset to work with real dates

This is a dataset of patient visits from a medical office. It contains the patients first and last names, date of birth, and the dates of their first 3 visits. 

In [4]:
from pyspark.sql.types import *

md_office = [('Mohammed','Alfasy','1987-4-8','2016-1-7','2017-2-3','2018-3-2') \
            ,('Marcy','Wellmaker','1986-4-8','2015-1-7','2017-1-3','2018-1-2') \
            ,('Ginny','Ginger','1986-7-10','2014-8-7','2015-2-3','2016-3-2') \
            ,('Vijay','Doberson','1988-5-2','2016-1-7','2018-2-3','2018-3-2') \
            ,('Orhan','Gelicek','1987-5-11','2016-5-7','2017-1-3','2018-9-2') \
            ,('Sarah','Jones','1956-7-6','2016-4-7','2017-8-3','2018-10-2') \
            ,('John','Johnson','2017-10-12','2018-1-2','2018-10-3','2018-3-2') ]

df = spark.createDataFrame(md_office,['first_name','last_name','dob','visit1','visit2','visit3']) # schema=final_struc

# Check to make sure it worked
df.show()
print(df.printSchema())

+----------+---------+----------+--------+---------+---------+
|first_name|last_name|       dob|  visit1|   visit2|   visit3|
+----------+---------+----------+--------+---------+---------+
|  Mohammed|   Alfasy|  1987-4-8|2016-1-7| 2017-2-3| 2018-3-2|
|     Marcy|Wellmaker|  1986-4-8|2015-1-7| 2017-1-3| 2018-1-2|
|     Ginny|   Ginger| 1986-7-10|2014-8-7| 2015-2-3| 2016-3-2|
|     Vijay| Doberson|  1988-5-2|2016-1-7| 2018-2-3| 2018-3-2|
|     Orhan|  Gelicek| 1987-5-11|2016-5-7| 2017-1-3| 2018-9-2|
|     Sarah|    Jones|  1956-7-6|2016-4-7| 2017-8-3|2018-10-2|
|      John|  Johnson|2017-10-12|2018-1-2|2018-10-3| 2018-3-2|
+----------+---------+----------+--------+---------+---------+

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- visit1: string (nullable = true)
 |-- visit2: string (nullable = true)
 |-- visit3: string (nullable = true)

None


Oh no! The dates are still stored as text... let's try converting them again and see if we have any issues this time.

In [6]:
from pyspark.sql.functions import * 

from pyspark.sql.types import *
from pyspark.sql.functions import expr

df_new= df.withColumn("dob",to_date(df.visit1))\
            .withColumn("visit1", to_date(df.visit1))\
            .withColumn("visit2", to_date(df.visit2))\
            .withColumn("visit3", to_date(df.visit3))
#df_new= spark.sql("select to_date(dob,'MM-dd-yyyy') date").show()
df_new.limit(4).toPandas()


,first_name,last_name,dob,visit1,visit2,visit3
0,Mohammed,Alfasy,2016-01-07,2016-01-07,2017-02-03,2018-03-02
1,Marcy,Wellmaker,2015-01-07,2015-01-07,2017-01-03,2018-01-02
2,Ginny,Ginger,2014-08-07,2014-08-07,2015-02-03,2016-03-02
3,Vijay,Doberson,2016-01-07,2016-01-07,2018-02-03,2018-03-02


In [7]:
df_new.printSchema()

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- visit1: date (nullable = true)
 |-- visit2: date (nullable = true)
 |-- visit3: date (nullable = true)



## 7. Can you calculate a variable showing the length of time between patient visits?

Compare visit1 to visit2 and visit2 to visit3 for all patients and see what the average length of time is between visits. Create an alias for it as well. 

In [8]:
df_new= df_new.withColumn('diffv12',datediff(df_new.visit2,df_new.visit1))\
                .withColumn('diffv23',datediff(df_new.visit3,df_new.visit2))
df_new.limit(4).toPandas()

,first_name,last_name,dob,visit1,visit2,visit3,diffv12,diffv23
0,Mohammed,Alfasy,2016-01-07,2016-01-07,2017-02-03,2018-03-02,393,392
1,Marcy,Wellmaker,2015-01-07,2015-01-07,2017-01-03,2018-01-02,727,364
2,Ginny,Ginger,2014-08-07,2014-08-07,2015-02-03,2016-03-02,180,393
3,Vijay,Doberson,2016-01-07,2016-01-07,2018-02-03,2018-03-02,758,27


In [14]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

# avg_col = udf(lambda x,y: sum(array)/2, DoubleType())
# df_new.withColumn("average_time_between_visits", avg_col(list("diffv12", "diffv23"))).limit(4).toPandas()

## 8. Can you calculate the age of each patient?

In [18]:
df_new= df_new.withColumn('Age',datediff(df_new.visit3,df_new.dob)/365.25)
df_new.limit(4).toPandas()

,first_name,last_name,dob,visit1,visit2,visit3,diffv12,diffv23,Age
0,Mohammed,Alfasy,2016-01-07,2016-01-07,2017-02-03,2018-03-02,393,392,2.149213
1,Marcy,Wellmaker,2015-01-07,2015-01-07,2017-01-03,2018-01-02,727,364,2.986995
2,Ginny,Ginger,2014-08-07,2014-08-07,2015-02-03,2016-03-02,180,393,1.568789
3,Vijay,Doberson,2016-01-07,2016-01-07,2018-02-03,2018-03-02,758,27,2.149213


## 9. Can you extract the month from the visit columns and call it "Month"?

In [19]:
df_new= df_new.withColumn('Month',month(df_new.visit3))
df_new.limit(4).toPandas()

,first_name,last_name,dob,visit1,visit2,visit3,diffv12,diffv23,Age,Month
0,Mohammed,Alfasy,2016-01-07,2016-01-07,2017-02-03,2018-03-02,393,392,2.149213,3
1,Marcy,Wellmaker,2015-01-07,2015-01-07,2017-01-03,2018-01-02,727,364,2.986995,1
2,Ginny,Ginger,2014-08-07,2014-08-07,2015-02-03,2016-03-02,180,393,1.568789,3
3,Vijay,Doberson,2016-01-07,2016-01-07,2018-02-03,2018-03-02,758,27,2.149213,3


## 10. Challenges with working with date and timestamps

Let's read in the supermarket sales dataframe attached to the lecture now and see some of the issues that can come up when working with date and timestamps values.

In [33]:
supermarket = spark.read.csv('Datasets/supermarket_sales.csv',inferSchema=True,header=True)

## About this dataset

The growth of supermarkets in most populated cities are increasing and market competitions are also high. The dataset is one of the historical sales of supermarket company which has recorded in 3 different branches for 3 months data. 

 - Attribute information
 - Invoice id: Computer generated sales slip invoice identification number
 - Branch: Branch of supercenter (3 branches are available identified by A, B and C).
 - City: Location of supercenters
 - Customer type: Type of customers, recorded by Members for customers using member card and Normal for without member card.
 - Gender: Gender type of customer
 - Product line: General item categorization groups - Electronic accessories, Fashion accessories, Food and beverages, Health and beauty, Home and lifestyle, Sports and travel
 - Unit price: Price of each product in USD
 - Quantity: Number of products purchased by customer
 - Tax: 5% tax fee for customer buying
 - Total: Total price including tax
 - Date: Date of purchase (Record available from January 2019 to March 2019)
 - Time: Purchase time (10am to 9pm)
 - Payment: Payment used by customer for purchase (3 methods are available – Cash, Credit card and Ewallet)
 - COGS: Cost of goods sold
 - Gross margin percentage: Gross margin percentage
 - Gross income: Gross income
 - Rating: Customer stratification rating on their overall shopping experience (On a scale of 1 to 10)

**Source:** https://www.kaggle.com/aungpyaeap/supermarket-sales

### View dataframe and schema as usual

In [21]:
supermarket.limit(4).toPandas()

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,2023-02-10 13:08:00,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,2023-02-10 10:29:00,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,2023-02-10 13:23:00,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,2023-02-10 20:33:00,Ewallet,465.76,4.761905,23.2880,8.4


In [22]:
supermarket.printSchema()

root
 |-- Invoice ID: string (nullable = true)
 |-- Branch: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Customer type: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- Unit price: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Tax 5%: double (nullable = true)
 |-- Total: double (nullable = true)
 |-- Date: string (nullable = true)
 |-- Time: timestamp (nullable = true)
 |-- Payment: string (nullable = true)
 |-- cogs: double (nullable = true)
 |-- gross margin percentage: double (nullable = true)
 |-- gross income: double (nullable = true)
 |-- Rating: double (nullable = true)



### Convert date field to date type

Looks like we need to convert the date field into a date type. Let's go ahead and do that..

In [31]:
# from pyspark.sql.functions import regexp_replace, regexp_extract
# supermarket=supermarket.withColumn("Date",regexp_replace(supermarket.Date, '/', '-'))
# supermarket.limit(4).toPandas()


In [34]:
df_new= supermarket.withColumn("Date",to_date(supermarket.Date))
df_new.limit(4).toPandas()

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,None,2023-02-10 13:08:00,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,None,2023-02-10 10:29:00,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,None,2023-02-10 13:23:00,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,None,2023-02-10 20:33:00,Ewallet,465.76,4.761905,23.2880,8.4


### How can we extract the month value from the date field?

If you had trouble converting the date field in the previous question think about a more creative solution to extract the month from that field.

## 11.0 Working with Arrays

Here is a dataframe of reviews from the movie the Dark Night.

In [50]:
from pyspark.sql.functions import *

values = [(5,'Epic. This is the best movie I have EVER seen'), \
          (4,'Pretty good, but I would have liked to seen better special effects'), \
          (3,'So so. Casting could have been improved'), \
          (5,'The most EPIC movie of the year! Casting was awesome. Special effects were so intense.'), \
          (4,'Solid but I would have liked to see more of the love story'), \
          (5,'THE BOMB!!!!!!!')]
reviews = spark.createDataFrame(values,['rating', 'review_txt'])

reviews.show(6,False)

+------+--------------------------------------------------------------------------------------+
|rating|review_txt                                                                            |
+------+--------------------------------------------------------------------------------------+
|5     |Epic. This is the best movie I have EVER seen                                         |
|4     |Pretty good, but I would have liked to seen better special effects                    |
|3     |So so. Casting could have been improved                                               |
|5     |The most EPIC movie of the year! Casting was awesome. Special effects were so intense.|
|4     |Solid but I would have liked to see more of the love story                            |
|5     |THE BOMB!!!!!!!                                                                       |
+------+--------------------------------------------------------------------------------------+



## 11.1 Let's see if we can create an array off of the review text column and then derive some meaningful results from it.

**But first** we need to clean the rview_txt column to make sure we can get what we need from our analysis later on. So let's do the following:

1. Remove all punctuation
2. lower case everything
3. Remove white space (trim)
3. Then finally, split the string

In [56]:
array_df = reviews.select("rating",split(reviews.review_txt, ' ').alias('title_array'))
array_df.show(6,False)

+------+------------------------------------------------------------------------------------------------------+
|rating|title_array                                                                                           |
+------+------------------------------------------------------------------------------------------------------+
|5     |[Epic., This, is, the, best, movie, I, have, EVER, seen]                                              |
|4     |[Pretty, good,, but, I, would, have, liked, to, seen, better, special, effects]                       |
|3     |[So, so., Casting, could, have, been, improved]                                                       |
|5     |[The, most, EPIC, movie, of, the, year!, Casting, was, awesome., Special, effects, were, so, intense.]|
|4     |[Solid, but, I, would, have, liked, to, see, more, of, the, love, story]                              |
|5     |[THE, BOMB!!!!!!!]                                                                              

In [52]:
array_df.select(array_remove(array_df.title_array, '.')).show(5, False)

+------------------------------------------------------------------------------------------------------+
|array_remove(title_array, .)                                                                          |
+------------------------------------------------------------------------------------------------------+
|[Epic., This, is, the, best, movie, I, have, EVER, seen]                                              |
|[Pretty, good,, but, I, would, have, liked, to, seen, better, special, effects]                       |
|[So, so., Casting, could, have, been, improved]                                                       |
|[The, most, EPIC, movie, of, the, year!, Casting, was, awesome., Special, effects, were, so, intense.]|
|[Solid, but, I, would, have, liked, to, see, more, of, the, love, story]                              |
+------------------------------------------------------------------------------------------------------+
only showing top 5 rows



In [61]:
import pyspark.sql.functions as F
array_df.withColumn('title_array',F.transform(F.col('title_array'), lambda x:F.lower(x))).show(5,False)

+------+------------------------------------------------------------------------------------------------------+
|rating|title_array                                                                                           |
+------+------------------------------------------------------------------------------------------------------+
|5     |[epic., this, is, the, best, movie, i, have, ever, seen]                                              |
|4     |[pretty, good,, but, i, would, have, liked, to, seen, better, special, effects]                       |
|3     |[so, so., casting, could, have, been, improved]                                                       |
|5     |[the, most, epic, movie, of, the, year!, casting, was, awesome., special, effects, were, so, intense.]|
|4     |[solid, but, i, would, have, liked, to, see, more, of, the, love, story]                              |
+------+------------------------------------------------------------------------------------------------

In [66]:
array_df.withColumn('title_array',F.transform(F.col('title_array'), lambda x: F.trim(x))).show(5,False)

+------+------------------------------------------------------------------------------------------------------+
|rating|title_array                                                                                           |
+------+------------------------------------------------------------------------------------------------------+
|5     |[Epic., This, is, the, best, movie, I, have, EVER, seen]                                              |
|4     |[Pretty, good,, but, I, would, have, liked, to, seen, better, special, effects]                       |
|3     |[So, so., Casting, could, have, been, improved]                                                       |
|5     |[The, most, EPIC, movie, of, the, year!, Casting, was, awesome., Special, effects, were, so, intense.]|
|4     |[Solid, but, I, would, have, liked, to, see, more, of, the, love, story]                              |
+------+------------------------------------------------------------------------------------------------

## 11.2 Alright now let's see if we can find which reviews contain the word 'Epic'

In [44]:
reviews.where(reviews['review_txt'].like("%Epic%")).show(6,False)

+------+---------------------------------------------+
|rating|review_txt                                   |
+------+---------------------------------------------+
|5     |Epic. This is the best movie I have EVER seen|
+------+---------------------------------------------+



### That's it! Great Job!